In [1]:
import tensorflow as tf
import numpy as np

# Download the default Shakespeare dataset
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Read the text
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

print(f'Unique characters: {len(vocab)}')

Unique characters: 65


In [2]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    return chunk[:-1], chunk[1:]

dataset = sequences.map(split_input_target).shuffle(10000).batch(64, drop_remainder=True)

In [4]:
import tensorflow as tf

# Define parameters
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024
def build_flexible_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_flexible_model(vocab_size, embedding_dim, rnn_units)

# Compile
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:

model.fit(dataset, epochs=3)

Epoch 1/3
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 52ms/step - loss: 3.1332
Epoch 2/3
172/172 ━━━━━━━━━━━━━━━━━━━━ 11s 52ms/step - loss: 1.9120
Epoch 3/3
172/172 ━━━━━━━━━━━━━━━━━━━━ 11s 53ms/step - loss: 1.6223


In [7]:
def generate_text_simple(model, start_string, gen_chars=500):
    # Convert start string to numbers
    input_indices = [char2idx[s] for s in start_string]
    input_indices = tf.expand_dims(input_indices, 0) # Add batch dimension

    predictions_log = []

    for _ in range(gen_chars):
        predictions = model(input_indices)

        predictions = predictions[:, -1, :]

        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_indices = tf.concat([input_indices, [[predicted_id]]], axis=-1)
        predictions_log.append(idx2char[predicted_id])

    return start_string + ''.join(predictions_log)


print(generate_text_simple(model, start_string="ROMEO: "))

ROMEO: his an I nere?

QUEEN ELIZABETH:
That to have coveshing to me siet and
It you make as friends a faults of crossford.

PAPULET:
And fare shall wo dack my edeliable.

AGHONTES:
Welcome jeacent, she of the shuws;
To cantit winter thy strange: that, thur stand,
It is depored and quacest thus well.
That reped my hearth in alms.

RLOY BETHONGO:
Not whence king in this stoon's beather, sines signify
Be fauth him stait dryat tolve
of mine, chores encolp is me to be abres have so dear stright;
While his 
